- [x] Try out divergence package!
- Problem with divergence package: needs arrayfire. Might be possible to reimplement this, but cocos does not run on mac. If we really want this, might be easiest to install it on the compute cluster.

- Torch actually has a pretty nice implementation of kl_divergence (that is only implemented for analytical stuff of course)


In [41]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import hvplot.pandas
import adaptive_nof1
import numpy
from adaptive_nof1.policies import ThompsonSampling
from adaptive_nof1.inference import *
from adaptive_nof1.series_of_simulations_runner import simulate_configurations
from adaptive_nof1.helpers import *
from adaptive_nof1.models import SelfExperimentationModel
import pandas
import matplotlib.pyplot as plt
import holoviews
import panel
import torch
import arviz

In [227]:
# Setup a simple pymc model

observed = [0.1, 0.2, 0.3]

trace_1 = None
trace_2 = None
model_1 = pymc.Model()
with model_1:
    mu = pymc.Normal("mu_0", mu=1, sigma=1)
    outcome = pymc.Normal(
        "outcome",
        mu=mu,
        observed=observed,
        sigma=1,
    )
    trace_1 = pymc.sample(200, progressbar=False)

model_2 = pymc.Model()
with model_2:
    mu = pymc.Normal("mu_0", mu=5, sigma=1)
    outcome = pymc.Normal(
        "outcome",
        mu=mu,
        observed=observed,
        sigma=1,
    )
    trace_2 = pymc.sample(200, progressbar=False)

with model_2:
    pymc.compute_log_likelihood(trace_1)
    pymc.compute_log_likelihood(trace_2)

Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_0]
Sampling 4 chains for 1_000 tune and 200 draw iterations (4_000 + 800 draws total) took 0 seconds.
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu_0]
Sampling 4 chains for 1_000 tune and 200 draw iterations (4_000 + 800 draws total) took 0 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation.

In [19]:
print(trace_1.log_likelihood.mean()["outcome"].values)
print(trace_2.log_likelihood.mean()["outcome"].values)

-1.7399987822984488
-1.8322727042305325


In [20]:
samples = numpy.random.normal(0, 1, size=100)

In [21]:
samples

array([-0.30290411,  1.54043294, -1.5471573 , -0.42769506, -1.57965017,
       -0.66460858, -1.03118724, -0.36528467,  1.70443051,  1.23986521,
       -0.16452961,  1.67591376,  0.10041071,  0.89402071, -2.2761129 ,
        0.08574462, -3.02841481, -0.1035435 , -0.31709598, -0.34835679,
        0.08814709,  1.04918378,  1.0712992 ,  1.21248814,  1.71076498,
        0.25661288, -0.99690436, -0.56575434,  0.46756592, -0.72520482,
        2.01638103,  0.1497881 , -0.53484392,  1.67978841,  1.73375149,
        0.42342362, -0.78185611, -1.3705416 ,  1.29680064, -1.15937866,
        2.0424563 ,  1.25902875,  0.24910558,  0.92008841, -0.39096747,
        0.8237869 , -0.10797054,  0.44237897, -0.09138614,  0.71460833,
        1.07927523,  0.09663729,  0.12653346,  0.90001616, -0.7293588 ,
       -0.76472118, -0.42309328, -0.30684041, -0.29716589,  0.06212765,
       -1.89546099,  0.1190043 ,  0.39818323, -0.48553131,  0.58940148,
        0.92437546, -0.5682202 , -0.26803859, -0.6016287 ,  0.91

In [226]:
for m_0, m_1 in [[0.0, 0.0], [1.0, 0.0], [0.0, 1.0], [10.0, 20.0], [1.0, 5.0]]:
    analytical_kl_divergence = (1 + (m_0 - m_1) ** 2) / 2 - 0.5
    # model_1 = pymc.Model()
    # with model_1:
    #    mu_0 = pymc.Normal("mu_0", m_0, sigma=1)
    #    outcome = pymc.Normal("outcome", mu=m_0, sigma=1, observed=observed)
    #    trace_1 = pymc.sample()
    #
    # model_2 = pymc.Model()
    # with model_2:
    #     mu_1 = pymc.Normal("mu_0", m_1, sigma=1)
    #     outcome = pymc.Normal("outcome", mu=m_1, sigma=1, observed=observed)
    #     trace_2 = pymc.sample()

    # with model_1:
    #     llk = pymc.Deterministic('llk', model_1.logp())
    #     logp = pymc.sample_posterior_predictive(trace_2, var_names=["llk"])
    #     h_qp = logp.posterior_predictive["llk"].mean()

    # with model_2:
    #     llk = pymc.Deterministic('llk', model_2.logp())
    #     logp = pymc.sample_posterior_predictive(trace_2, var_names=["llk"])
    #     h_qp = logp.posterior_predictive["llk"].mean()

    print(m_0, m_1)
    m_t1 = torch.distributions.Normal(torch.tensor([m_0]), torch.tensor([1.0]))
    m_t2 = torch.distributions.Normal(torch.tensor([m_1]), torch.tensor([1.0]))

    print(
        f"analytical: {analytical_kl_divergence}, torch: {torch.distributions.kl_divergence(m_t1, m_t2)}"
    )

0.0 0.0
analytical: 0.0, torch: tensor([0.])
1.0 0.0
analytical: 0.5, torch: tensor([0.5000])
0.0 1.0
analytical: 0.5, torch: tensor([0.5000])
10.0 20.0
analytical: 50.0, torch: tensor([50.])
1.0 5.0
analytical: 8.0, torch: tensor([8.])


In [311]:
from scipy.stats import norm

norm_5 = norm(5, 1)
norm_1 = norm(1, 1)

norm_2_3 = norm(2, 3)
norm_1_2 = norm(1, 2)

# m = MultivariateNormal(torch.zeros(2), torch.eye(2))

# m_1 = Normal(torch.tensor([1.0]), torch.tensor([1.0]))
# m_5 = Normal(torch.tensor([5.0]), torch.tensor([1.0]))

print(kl(norm_1.rvs(size=10000), norm_1, norm_5))
print(kl(norm_5.rvs(size=10000), norm_5, norm_1))

NameError: name 'kl' is not defined

In [252]:
norm_5.rvs(size=10)

array([3.6491356 , 5.17315229, 4.91241923, 4.60486837, 2.88127543,
       4.12525632, 4.31532115, 4.91923265, 4.27691579, 4.76410442])

In [253]:
(1 + (5 - 1) ** 2) / 2 + -0.5

8.0

In [8]:
relative_entropy_from_samples(
    norm_1.rvs(size=10000), norm_5.rvs(size=10000), discrete=False
)

NameError: name 'relative_entropy_from_samples' is not defined

In [35]:
# Numerical integration approach

from sklearn.neighbors import KernelDensity

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])


# this outputs the log_likelihood per sample
print(kde.score_samples([[1, 2], [3, 4]]))

# we can also just print the log likelihood
print(kde.score([[1, 2], [3, 4]]))

# Let's try with the trace data

[-3.52404094 -5.57052263]
-9.094563568086873


In [93]:
X = norm_1.rvs(size=1000).reshape(-1, 1)
Y = norm_5.rvs(size=1000).reshape(-1, 1)
kde = KernelDensity(kernel="gaussian").fit(X)

[[ 8.21658522e-01]
 [ 2.47903629e-01]
 [ 1.81141233e+00]
 [ 1.50889372e-01]
 [ 3.61912452e+00]
 [ 1.63162412e+00]
 [-7.50494556e-01]
 [ 2.58932050e+00]
 [ 2.30373455e+00]
 [ 2.99862272e-01]
 [ 8.79817523e-01]
 [ 2.31289950e+00]
 [-1.42049856e+00]
 [ 1.45325301e+00]
 [ 2.34794015e+00]
 [ 2.25569991e+00]
 [ 1.26679999e+00]
 [-6.68511546e-01]
 [ 1.28628717e+00]
 [ 1.46003595e+00]
 [ 1.11290662e+00]
 [ 4.38111447e-01]
 [ 7.45182648e-01]
 [ 2.65346997e+00]
 [ 1.53780806e+00]
 [ 7.18893128e-01]
 [ 1.24279079e+00]
 [ 1.54635575e+00]
 [ 1.89486045e+00]
 [ 2.06839774e+00]
 [ 4.44777366e-01]
 [-1.69438357e-01]
 [ 1.03327245e+00]
 [ 6.69778155e-01]
 [ 1.53598610e+00]
 [ 1.89154681e+00]
 [ 1.44085569e+00]
 [-3.43117090e-01]
 [ 2.11392225e+00]
 [ 2.08667566e+00]
 [ 1.37846901e+00]
 [ 1.69111487e+00]
 [ 1.84719751e+00]
 [ 1.86259333e+00]
 [ 5.53641940e-01]
 [ 2.26909464e+00]
 [ 1.33699929e+00]
 [ 1.61229217e-01]
 [ 2.62255280e-01]
 [ 2.69499317e+00]
 [-9.81685844e-01]
 [ 7.24812648e-01]
 [ 2.6393172

In [92]:
print(kde.score(X))
print(kde.score(Y))


kde.score_samples(Y).mean() - kde.score_samples(X).mean()

-1531.9406987606972
-5397.361494542729


-3.8654207957820317

In [383]:
import typing as tp
import statsmodels.api as sm
from cubature import cubature


def _select_vectorized_log_fun_for_base(base: float, gpu: bool = False) -> tp.Callable:
    if base == 2:
        if gpu:
            return cn.log2
        else:
            return np.log2
    if base == np.e:
        if gpu:
            return cn.log
        else:
            return np.log
    if base == 10:
        if gpu:
            return cn.log10
        else:
            return np.log10

    raise ValueError("base not supported")


def intersection(
    a0: float, b0: float, a1: float, b1: float
) -> tp.Optional[tp.Tuple[float, float]]:
    """
    Calculate the intersection of two intervals [a0, b0] and [a1, b1]. If the intervals do not
    overlap the function returns None. The parameters must satisfy a0 <= b0 and a1 <= b1.

    Parameters
    ----------
    a0: beginning of the first interval
    b0: end of the first interval
    a1: beginning of the second interval
    b1: end of the second interval

    Returns
    -------

    """
    assert a0 <= b0
    assert a1 <= b1

    if a0 >= b1:
        return None

    if b0 < a1:
        return None

    return max(a0, a1), min(b0, b1)


def _does_support_overlap(
    p: sm.nonparametric.KDEUnivariate, q: sm.nonparametric.KDEUnivariate
) -> bool:
    """
    Determine whether the support of distributions of kernel density estimates p and q overlap.

    Parameters
    ----------
    p: statsmodels kde object representing an approximation of the distribution p
    q: statsmodels kde object representing an approximation of the distribution q

    Returns
    -------
    whether the support of distributions of kernel density estimates p and q overlap
    """
    return (
        intersection(min(p.support), max(p.support), min(q.support), max(q.support))
        is not None
    )


################################################################################
# Relative Entropy (KL Divergence)
################################################################################
def _relative_entropy_integrand(
    p: tp.Callable, q: tp.Callable, x: float, log_fun: tp.Callable = np.log
) -> float:
    """
    Compute the integrand p(x) * log(p(x) / q(x)) at a given point x for the calculation of relative
    entropy.

    Parameters
    ----------
    p: probability density function of the distribution p
    q: probability density function of the distribution q
    x: the point at which to evaluate the integrand
    base: the base of the logarithm used to control the units of measurement for the result

    Returns
    -------
    Integrand for the relative entropy calculation
    """
    qx = q(x)
    px = p(x)
    if qx == 0.0:
        if px == 0.0:
            return 0.0
        else:
            raise ValueError(f"q(x) is zero at x={x} but p(x) is not")
    elif px == 0.0:
        return 0.0
    else:
        return px * log_fun(px / qx)


def _vectorized_relative_entropy_integrand(
    p: tp.Callable, q: tp.Callable, x: np.ndarray, log_fun: tp.Callable = np.log
) -> np.ndarray:
    """
    Compute the integrand p(x) * log(p(x) / q(x)) vectorized at given points x for the calculation
    of relative entropy.

    Parameters
    ----------
    p: probability density function of the distribution p
    q: probability density function of the distribution q
    x: the point at which to evaluate the integrand
    base: the base of the logarithm used to control the units of measurement for the result

    Returns
    -------
    Integrand for the cross entropy calculation
    """
    # return p(x) * log_fun(q(x) + 1e-12)
    qx = q(x)
    px = p(x)

    q_positive_index = qx > 0.0
    p_positive_index = px > 0.0

    q_zero_but_p_positive_index = ~q_positive_index & p_positive_index
    if np.any(q_zero_but_p_positive_index):
        raise ValueError(
            f"q(x) is zero at x={x[q_zero_but_p_positive_index]} but p(x) is not"
        )

    return np.where(p_positive_index, px * log_fun(px / qx), 0.0)


def relative_entropy_from_densities_with_support(
    p: tp.Callable,
    q: tp.Callable,
    a: float,
    b: float,
    base: float = np.e,
    eps_abs: float = 1.49e-08,
    eps_rel: float = 1.49e-08,
) -> float:
    """
    Compute the relative entropy of the distribution q relative to the distribution p

                D_KL(p||q) = E_p [log(p/q)]

    via numerical integration from a to b.
    The argument base can be used to specify the units in which the entropy is measured.
    The default choice is the natural logarithm.

    Parameters
    ----------
    p: probability density function of the distribution p
    q: probability density function of the distribution q
    a: lower bound of the integration region
    b: upper bound of the integration region
    base: the base of the logarithm used to control the units of measurement for the result
    eps_abs: absolute error tolerance for numerical integration
    eps_rel: relative error tolerance for numerical integration

    Returns
    -------
    The relative entropy of the distribution q relative to the distribution p.
    """
    log_fun = _select_vectorized_log_fun_for_base(base)

    def integrand(x: float):
        return _relative_entropy_integrand(p=p, q=q, x=x, log_fun=log_fun)

    return cubature(
        func=integrand,
        ndim=1,
        fdim=1,
        xmin=np.array([a]),
        xmax=np.array([b]),
        vectorized=False,
        adaptive="p",
        abserr=eps_abs,
        relerr=eps_rel,
    )[0].item()


def relative_entropy_from_kde(
    p, q, base: float = np.e, eps_abs: float = 1.49e-08, eps_rel: float = 1.49e-08
) -> float:
    """
    Compute the relative entropy of the distribution q relative to the distribution p

                D_KL(p||q) E_p [log(p/q)]

    given by the statsmodels kde objects via numerical integration.
    The argument base can be used to specify the units in which the entropy is measured.
    The default choice is the natural logarithm.

    Parameters
    ----------
    p: statsmodels kde object approximating the probability density function of the distribution p
    q: statsmodels kde object approximating the probability density function of the distribution q
    base: the base of the logarithm used to control the units of measurement for the result
    eps_abs: absolute error tolerance for numerical integration
    eps_rel: relative error tolerance for numerical integration

    Returns
    -------
    The relative entropy of the distribution q relative to the distribution p.
    """
    if not _does_support_overlap(p, q):
        raise ValueError("The support of p and q does not overlap.")

    a = min(min(p.support), min(q.support))
    b = max(max(p.support), max(q.support))
    return relative_entropy_from_densities_with_support(
        p=p.evaluate,
        q=q.evaluate,
        a=a,
        b=b,
        base=base,
        eps_abs=eps_abs,
        eps_rel=eps_rel,
    )


def continuous_relative_entropy_from_sample(
    sample_p: np.ndarray,
    sample_q: np.ndarray,
    base: float = np.e,
    eps_abs: float = 1.49e-08,
    eps_rel: float = 1.49e-08,
) -> float:
    """
    Compute the relative entropy of the distribution q relative to the distribution p

                D_KL(p||q) = E_p [log(p/q)]

    from samples of the two distributions via approximation by a kernel density estimate and
    numerical integration.
    The argument base can be used to specify the units in which the entropy is measured.
    The default choice is the natural logarithm.

    Parameters
    ----------
    sample_p: sample from the distribution p
    sample_q: sample from the distribution q
    base: the base of the logarithm used to control the units of measurement for the result
    eps_abs: absolute error tolerance for numerical integration
    eps_rel: relative error tolerance for numerical integration

    Returns
    -------
    The relative entropy of the distribution q relative to the distribution p.
    """
    kde_p = sm.nonparametric.KDEUnivariate(sample_p)
    kde_p.fit()
    kde_q = sm.nonparametric.KDEUnivariate(sample_q)
    kde_q.fit()

    return relative_entropy_from_kde(
        p=kde_p, q=kde_q, base=base, eps_abs=eps_abs, eps_rel=eps_rel
    )

In [384]:
X = norm_1.rvs(size=10000)
Y = norm_5.rvs(size=10000)
continuous_relative_entropy_from_sample(sample_p=X, sample_q=Y)

13.066778150554041

In [385]:
numpy.array(arviz.extract(trace_1.posterior)["mu_0"])

array([ 0.82911629,  1.41846731,  0.95478567,  0.72397859,  1.13860672,
       -0.338256  ,  0.11905979,  0.17342349, -0.21618747, -0.5623193 ,
        0.36551644, -0.007693  ,  0.79965197,  0.79965197,  0.8321046 ,
        0.78392357,  0.05367251, -0.42586403, -0.23620797,  0.17380338,
        0.32891852, -0.16932399,  0.21193245,  0.37689612, -0.86417899,
       -0.53253294,  0.28531581,  0.28531581,  0.48865781,  0.32016503,
        0.47549697,  0.3658043 , -1.50892222, -0.34388207,  0.23844604,
        0.22484007,  0.30058593,  0.23018652,  0.58471025,  0.8454085 ,
       -0.01442735,  0.83100777,  0.53172689,  0.23481492,  0.5580954 ,
        0.72917785,  0.07289284,  0.73928712,  0.92031338,  0.9205975 ,
        0.51696796,  0.51696796,  0.48068972,  1.40912773,  0.90845261,
        1.69458401,  0.45743682,  0.79590585,  0.51674995,  0.80397968,
        0.68970478,  0.44885373,  0.52978365,  0.25234241,  0.53046119,
        0.28176353,  0.28176353,  0.20089782,  0.28681086, -0.08

In [402]:
p_samples = norm_2_3.rvs(size=1000000)
q_samples = norm_1_2.rvs(size=1000000)

kde_p = sm.nonparametric.KDEUnivariate(p_samples)
kde_p.fit()
kde_q = sm.nonparametric.KDEUnivariate(q_samples)
kde_q.fit()

In [403]:
relative_entropy_from_kde(p=kde_p, q=kde_q, base=np.e)

ValueError: q(x) is zero at x=[17.45634889] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[17.45634889] but p(x) is not


ValueError: q(x) is zero at x=[16.27833883] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[16.27833883] but p(x) is not


ValueError: q(x) is zero at x=[17.15898954] but p(x) is not

...

ValueError: q(x) is zero at x=[15.71750437] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[15.71750437] but p(x) is not


ValueError: q(x) is zero at x=[15.53845899] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[15.53845899] but p(x) is not
/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py:99: RuntimeWarning: overflow encountered in divide
  return px * log_fun(px / qx)


inf

In [406]:
# calculate it with the real pdf to check if kde is the problem

p_min = min(kde_p.support)
p_max = max(kde_p.support)
q_min = min(kde_q.support)
q_max = max(kde_q.support)
combined_min = min(p_min, q_min)
combined_max = max(p_max, q_max)

relative_entropy_from_densities_with_support(
    norm_2_3.pdf, norm_1_2.pdf, combined_min, combined_max
)

0.3445304859557871

In [407]:
# calculate it with the pdf from scipy
from scipy import stats

kernel_p = stats.gaussian_kde(p_samples)
kernel_q = stats.gaussian_kde(q_samples)

p_min = min(kde_p.support)
p_max = max(kde_p.support)
q_min = min(kde_q.support)
q_max = max(kde_q.support)
combined_min = min(p_min, q_min)
combined_max = max(p_max, q_max)

relative_entropy_from_densities_with_support(
    kernel_p.evaluate, kernel_q.evaluate, combined_min, combined_max
)

ValueError: q(x) is zero at x=[17.45634889] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[17.45634889] but p(x) is not


ValueError: q(x) is zero at x=[16.27833883] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[16.27833883] but p(x) is not


ValueError: q(x) is zero at x=[17.15898954] but p(x) is not

...

ValueError: q(x) is zero at x=[15.80393067] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[15.80393067] but p(x) is not


ValueError: q(x) is zero at x=[15.44586687] but p(x) is not

Exception ignored in: 'cubature._cubature.integrand_wrapper'
Traceback (most recent call last):
  File "cubature/_cubature.pyx", line 57, in cubature._cubature.Integrand._call
  File "cubature/_cubature.pyx", line 48, in cubature._cubature.Integrand._call
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 169, in integrand
  File "/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py", line 95, in _relative_entropy_integrand
ValueError: q(x) is zero at x=[15.44586687] but p(x) is not
/var/folders/2g/v44yvb1n6sdgnp5mwbh8_qgc0000gn/T/ipykernel_9882/2639852434.py:99: RuntimeWarning: overflow encountered in divide
  return px * log_fun(px / qx)


inf